In [ ]:
#!unzip "/content/drive/MyDrive/Colab Notebooks/wastedetect.zip" -d "/content/drive/MyDrive/Colab Notebooks/wastedetect"

For object Detection and cropping the image from that result we are training a yolo model (yolov8n) which is used for detecting objects

This model requires a good GPU support to train faster, on CPU it can crash

mounting Google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


.




**Installing requirements **

In [2]:
!pip install gradio ultralytics Pillow
!pip install ultralytics roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

.


**GPU check**

In [3]:
import torch

print("✅ GPU available:", torch.cuda.is_available())

✅ GPU available: True


Training Yolo model with Epochs= 30, image input size= 416, Batch = 16 for avoiding GPU Crashes

In [4]:
from ultralytics import YOLO
import torch

# Confirm GPU
print("✅ GPU available:", torch.cuda.is_available())
print("🎯 Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

# Load YOLOv8 Small (better accuracy than 'nano', still fits T4)
model = YOLO("yolov8n.pt")

# Train only (no real validation used)
model.train(
    data="/content/drive/MyDrive/Colab Notebooks/wastedetect/data.yaml",
    epochs=30,
    imgsz=512,
    batch=16,
    workers=2,
    device=0,
    freeze=0,
    lr0=0.001,
    patience=15,
    optimizer="Adam",
    cos_lr=True,
    mosaic=1.0,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    translate=0.1, scale=0.5, shear=0.2,
    fliplr=0.5, flipud=0.0,
    mixup=0.1,
    cache=True,
    name="custom_yolo_colab_trainonly44"
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ GPU available: True
🎯 Device: Tesla T4


100%|██████████| 6.25M/6.25M [00:00<00:00, 110MB/s]


Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/Colab Notebooks/wastedetect/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=0, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=custom_yolo_colab_trainonly44, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=15, pe

100%|██████████| 755k/755k [00:00<00:00, 26.2MB/s]

Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all       4114       7889      0.935      0.923      0.967      0.868



30 epochs completed in 0.595 hours.
Optimizer stripped from runs/detect/custom_yolo_colab_trainonly44/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/custom_yolo_colab_trainonly44/weights/best.pt, 6.2MB

Validating runs/detect/custom_yolo_colab_trainonly44/weights/best.pt...
Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 129/129 [00:24<00:00,  5.25it/s]


                   all       4114       7889      0.933      0.924      0.968       0.87
             cardboard        762       1316      0.955      0.975      0.986      0.904
                 glass        720       1321      0.913      0.868      0.949      0.823
                 metal        604       1322      0.904      0.879      0.941      0.835
                 paper        345       1318      0.916      0.923       0.97      0.845
               plastic        958       1309      0.981       0.97       0.99       0.94
                 trash        753       1303      0.932      0.931      0.973      0.871
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/custom_yolo_colab_trainonly44


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c0dbc0bda90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

.


Loading and saving the model

In [7]:
!cp "/content/runs/detect/custom_yolo_colab_trainonly44/weights/last.pt" "/content/drive/MyDrive/Colab Notebooks/modelsave/last.pt"

In [ ]:
# Load model (from training run or pre-trained)
model = YOLO("/content/drive/MyDrive/Colab Notebooks/modelsave/best.pt")  # or any other path

# Save the model to a new location
model.save("/content/drive/MyDrive/Colab Notebooks/modelsave/best.pt")

.



making the final project from Yolov8n and EfficintnetV2S models
yolo will crop the picture and based in very big background but if yolo model becomes unable to detect the object it will pass the input to efficientnet model


In [ ]:
import gradio as gr
from ultralytics import YOLO
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
import time # Import the time module for delays

# --- Configuration for YOLOv8 Object Detection ---
CLASS_NAMES_YOLO = ['Paper', 'Cardboard', 'Plastic', 'Metal', 'Trash', 'Glass']
MODEL_PATH_YOLO = '/content/drive/MyDrive/Colab Notebooks/modelsave/best.pt'
PADDING_PIXELS = 20

# --- Configuration for EfficientNetV2S Classification ---
MODEL_PATH_CLASSIFICATION = '/content/drive/MyDrive/Colab Notebooks/modelsave/224EfficientNetV2S.keras'
CLASS_NAMES_CLASSIFICATION = ['Cardboard', 'Glass', 'Metal', 'Paper', 'Plastic', 'Trash']

# --- Load YOLOv8 Model ---
try:
    yolo_model = YOLO(MODEL_PATH_YOLO)
    print(f"YOLOv8 model loaded successfully from {MODEL_PATH_YOLO}")
except Exception as e:
    print(f"Error loading YOLOv8 model from {MODEL_PATH_YOLO}: {e}")
    print("Please ensure the YOLOv8 model path is correct and the model file exists.")
    exit()

# --- Load Classification Model ---
try:
    classification_model = tf.keras.models.load_model(MODEL_PATH_CLASSIFICATION)
    print(f"Classification model loaded successfully from {MODEL_PATH_CLASSIFICATION}")
except Exception as e:
    print(f"Error loading classification model from {MODEL_PATH_CLASSIFICATION}: {e}")
    print("Please ensure the Keras model path is correct and the model file exists.")
    exit()

# --- Classification Function for Cropped Images ---
def classify_cropped_image(img_np: np.ndarray) -> tuple[str, str, str]:
    """
    Classifies a single cropped image using the loaded EfficientNetV2S model.

    Args:
        img_np (np.ndarray): The input cropped image as a NumPy array.

    Returns:
        tuple[str, str, str]: A tuple containing:
            - str: A formatted string with the predicted class and confidence (e.g., "Paper (Conf: 0.95)").
            - str: An HTML string for the individual prediction status (e.g., "Predicted: Cardboard (Conf: 98.76%)").
            - str: The raw predicted label (e.g., "Cardboard").
    """
    img_pil = Image.fromarray(img_np.astype('uint8'))
    img_pil = img_pil.resize((224, 224))
    img_array = np.array(img_pil, dtype=np.float32)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    # Introduce a delay for EfficientNet classification
    time.sleep(0.5)

    prediction = classification_model.predict(img_array, verbose=0)
    predicted_index = np.argmax(prediction)

    classification_result_str = ""
    individual_prediction_html = ""
    predicted_label_raw = ""

    if predicted_index < len(CLASS_NAMES_CLASSIFICATION):
        predicted_label_raw = CLASS_NAMES_CLASSIFICATION[predicted_index]
        confidence = prediction[0][predicted_index]
        classification_result_str = f"{predicted_label_raw} (Conf: {confidence:.2f})"

        # CORRECTED: Used predicted_label_raw instead of undefined predicted_label
        individual_prediction_html = f"""
        <div style='text-align: center; font-size: 28px; font-weight: bold; color: #fbff00; font-family: Arial, sans-serif;'>
            Prediction: {predicted_label_raw}<br>
            <span style='font-size: 20px; color: #888;'>Confidence: {confidence:.2%}</span>
        </div>
        """
    else:
        classification_result_str = "Prediction Error"
        individual_prediction_html = f"""
        <div style='text-align: center; font-size: 20px; color: red; font-family: Arial, sans-serif;'>
            Classification Error: Class index {predicted_index} out of bounds.
        </div>
        """
    return classification_result_str, individual_prediction_html, predicted_label_raw


# --- Combined Object Detection and Classification Function ---
def detect_and_classify_objects(image_np: np.ndarray):
    """
    Performs object detection, crops detected objects, and then classifies each cropped object.
    If no objects are detected, it classifies the entire image. This is a generator function
    to provide immediate YOLO results and then progressive EfficientNet results.

    Args:
        image_np (np.ndarray): The input image frame as a NumPy array (H, W, C).

    Yields:
        tuple[np.ndarray, list, str]: A tuple containing:
            - np.ndarray: The image frame with bounding boxes drawn on it (or the original if no detections).
            - list: A list of (cropped_image_np, classification_label_string) tuples for the gallery.
            - str: An HTML string representing the final prediction status.
    """
    annotated_frame = image_np.copy()
    cropped_images_with_labels = []

    # 1. Perform YOLOv8 inference immediately
    results = yolo_model(image_np, conf=0.5, iou=0.45, verbose=False)

    detections_found = False
    all_boxes_data = [] # Store box data for drawing and later classification

    for r in results:
        if len(r.boxes) > 0:
            detections_found = True
            for box in r.boxes:
                all_boxes_data.append(box)
                x1_orig, y1_orig, x2_orig, y2_orig = map(int, box.xyxy[0])
                color = (0, 255, 0)
                cv2.rectangle(annotated_frame, (x1_orig, y1_orig), (x2_orig, y2_orig), color, 2)

    # Yield initial YOLO-only result
    initial_status = """
    <div style='text-align: center; font-size: 24px; font-weight: bold; color: #FFAA00; font-family: Arial, sans-serif;'>
        Classifying object...
    </div>
    """
    yield annotated_frame, [], initial_status # Yield annotated frame immediately

    # 2. Proceed with (delayed) EfficientNet classification
    final_prediction_status_html = "" # This will hold the HTML for the very last prediction

    if detections_found:
        for i, box in enumerate(all_boxes_data):
            x1_orig, y1_orig, x2_orig, y2_orig = map(int, box.xyxy[0])

            h, w, _ = image_np.shape
            x1_padded = max(0, x1_orig - PADDING_PIXELS)
            y1_padded = max(0, y1_orig - PADDING_PIXELS)
            x2_padded = min(w, x2_orig + PADDING_PIXELS)
            y2_padded = min(h, y2_orig + PADDING_PIXELS)

            if x2_padded > x1_padded and y2_padded > y1_padded:
                cropped_object_np = image_np[y1_padded:y2_padded, x1_padded:x2_padded]

                classification_result_str, individual_prediction_html, predicted_label = classify_cropped_image(cropped_object_np)

                # Update the final status with the latest classification
                # Use individual_prediction_html as the final status, as it contains the desired format
                final_prediction_status_html = individual_prediction_html

                # Add to gallery list - WITHOUT TEXT OVERLAY
                gallery_image = cropped_object_np.copy()
                cropped_images_with_labels.append((gallery_image, classification_result_str))

            # Yield results after each classification to show progress
            yield annotated_frame, cropped_images_with_labels, final_prediction_status_html

    else:
        # If no objects detected by YOLO, classify the entire image (after initial YOLO yield)
        print("No objects detected by YOLO. Classifying the entire image.")
        classification_result_str, individual_prediction_html, predicted_label = classify_cropped_image(image_np)

        # Use individual_prediction_html for full image classification status as well
        final_prediction_status_html = individual_prediction_html.replace("Prediction:", "Prediction (Full Image):") # Adjust text for full image

        # Add the full image with its classification to the gallery - WITHOUT TEXT OVERLAY
        gallery_image = image_np.copy()
        cropped_images_with_labels.append((gallery_image, "Full Image: " + classification_result_str))

        # Yield the final result for the full image classification
        yield annotated_frame, cropped_images_with_labels, final_prediction_status_html


# --- Gradio Interface Setup ---
iface = gr.Interface(
    fn=detect_and_classify_objects,
    inputs=gr.Image(type="numpy", label="Input Field"),
    outputs=[
        gr.Image(type="numpy", label="Detected Objects (Full Frame)"),
        gr.Gallery(label="Cropped Image Classification", columns=1, rows=1, object_fit="contain", height="auto"),
        gr.HTML(label="Latest Prediction Status")
    ],
    live=True,
    title="YOLOv8 Object Detection & EfficientNetV2S Classification",
    description="This application performs real-time object detection using YOLOv8, "
                "crops the detected objects, and then classifies each cropped object "
                "using an EfficientNetV2S model. If no objects are detected by YOLO, "
                "the entire frame will be classified by the EfficientNetV2S model. "
                f"YOLOv8 is configured to detect: {', '.join(CLASS_NAMES_YOLO)}. "
                f"The classification model predicts categories: {', '.join(CLASS_NAMES_CLASSIFICATION)}."
)

# --- Launch Gradio Interface ---
if __name__ == "__main__":
    print("Launching Gradio interface...")
    iface.queue()
    iface.launch()
    print("Gradio interface launched. Open the public URL in your browser.")

YOLOv8 model loaded successfully from /content/drive/MyDrive/Colab Notebooks/modelsave/best.pt
Classification model loaded successfully from /content/drive/MyDrive/Colab Notebooks/modelsave/224EfficientNetV2S.keras
Launching Gradio interface...
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3c6740c99ba36bc8f6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Gradio interface launched. Open the public URL in your browser.
